In [16]:
import cv2
import os, glob
import numpy as np
import json
import shutil
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
 
image_path = "data/temp"
point_path = "data/json"
save = "data/masks"
 
for p in os.listdir(point_path):
    masks = []
    with open(os.path.join(point_path, p), 'r') as f:
        data = json.load(f)
        img_name = p.split(".")[0]+".jpg"
        mask_name = p.split(".")[0]+"_mask.jpg"
        if not os.path.isfile(os.path.join(save, mask_name)):
            img = cv2.imread(os.path.join(image_path, img_name))
            cv2.imwrite(os.path.join("data/imgs", img_name), img)
            
            for d in data["shapes"]:
                # data = data["shapes"][0]["points"]
                data = d["points"]
                pts = np.array(data)
                pts = pts.astype(np.int32)
    
                mask = np.zeros(img.shape[:2], np.uint8)
                cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
                dst = cv2.bitwise_and(img, img, mask=mask)
    
                bg = np.zeros_like(img, np.uint8)*255
                # cv2.bitwise_not(bg, bg, mask=mask)
                res = bg + dst
                _, res = cv2.threshold(res, 1, 255, cv2.THRESH_BINARY)
                
                masks.append(res)
                
                if len(masks) > 1: masks[0] = cv2.bitwise_xor(masks[0], masks[-1])
                #     plt.subplot(1, 2, 1)
                #     plt.imshow(masks[0])
                #     plt.subplot(1, 2, 2)
                # plt.imshow(res)
                # plt.show()
            
            _, masks[0] = cv2.threshold(masks[0], 0, 255, cv2.THRESH_BINARY)
            cv2.imwrite(os.path.join(save, mask_name), masks[0])

3_01.jpg
3_02.jpg
4_01.jpg
4_02.jpg
5_01.jpg
5_02.jpg
6_01.jpg
6_02.jpg
8_01.jpg
8_02.jpg
